In [1]:
import pandas as pd
import numpy as np
import pickle
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import yfinance as yf

from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense, Dropout, GRU
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.sequence import TimeseriesGenerator


In [17]:
ticker = 'AAPL'

In [23]:
stocks = yf.Ticker(ticker).history(period='1y')
stocks = stocks.pct_change()
stocks[['Dividends','Stock Splits']] = stocks[['Dividends','Stock Splits']].fillna(0)
stocks.dropna(inplace=True)
stocks['target'] = (stocks['Close'] >= 0).astype(int)

Xb = stocks.drop(columns='target')
yb = stocks['target']
Xb_train, Xb_test, yb_train, yb_test = train_test_split(Xb, yb, shuffle=False)

ss = StandardScaler()
Xb_train_sc = ss.fit_transform(Xb_train)
Xb_test_sc = ss.transform(Xb_test)

trainb_seq = TimeseriesGenerator(Xb_train_sc, yb_train, length=3, batch_size=64)
print(f'train shape is {trainb_seq[0][0].shape}')
testb_seq = TimeseriesGenerator(Xb_test_sc, yb_test, length=3, batch_size=64)
print(f'test shape is {testb_seq[0][0].shape}')

train shape is (64, 3, 7)
test shape is (60, 3, 7)


In [24]:
base = Sequential()
base.add(GRU(8, input_shape=(3,7), return_sequences=True)) 
base.add(GRU(8, return_sequences=True)) 
base.add(Dense(8, activation='relu'))
base.add(Dense(1, activation='sigmoid'))

base.compile(optimizer=Adam(), loss='bce', metrics=['acc'])

In [25]:
histb = base.fit(trainb_seq, epochs=20, validation_data=testb_seq)

Epoch 1/20
3/3 [==============================] - 1s 296ms/step - loss: 0.6913 - acc: 0.5305 - val_loss: 0.6907 - val_acc: 0.5444
Epoch 2/20
3/3 [==============================] - 0s 25ms/step - loss: 0.6903 - acc: 0.5466 - val_loss: 0.6900 - val_acc: 0.5389
Epoch 3/20
3/3 [==============================] - 0s 27ms/step - loss: 0.6895 - acc: 0.5681 - val_loss: 0.6893 - val_acc: 0.5389
Epoch 4/20
3/3 [==============================] - 0s 24ms/step - loss: 0.6886 - acc: 0.5681 - val_loss: 0.6886 - val_acc: 0.5556
Epoch 5/20
3/3 [==============================] - 0s 26ms/step - loss: 0.6877 - acc: 0.5681 - val_loss: 0.6879 - val_acc: 0.5556
Epoch 6/20
3/3 [==============================] - 0s 24ms/step - loss: 0.6868 - acc: 0.5681 - val_loss: 0.6872 - val_acc: 0.5611
Epoch 7/20
3/3 [==============================] - 0s 26ms/step - loss: 0.6858 - acc: 0.5663 - val_loss: 0.6866 - val_acc: 0.5611
Epoch 8/20
3/3 [==============================] - 0s 24ms/step - loss: 0.6849 - acc: 0.5663 - va